# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

# Overview of the data

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [18]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [19]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


# Cleaning NHL_DF

In [20]:

nhl_df=nhl_df[nhl_df['year'] == 2018] # Teams appear several times, we should only include 2018 data
nhl_df['team'] = nhl_df['team'].str.replace(r'\*','')
cols = ['team', 'W', 'L']
nhl_df = nhl_df[cols]
nhl_df.head()
# NOTE: In nhl_df's data frame the team columns displays like this: '(Metropolitan Area) Team'

,team,W,L
0,Atlantic Division,Atlantic Division,Atlantic Division
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30


# Cleaning Cities

In [21]:
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]','')
cols = ['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']
cities = cities[cols]
cities.head()

,Metropolitan area,Population (2016 est.)[8],NHL
0,New York City,20153634,RangersIslandersDevils
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [33]:
nhl_df.head()

,team,W,L
0,Atlantic Division,Atlantic Division,Atlantic Division
1,Tampa Bay Lightning,54,23
2,Boston Bruins,50,20
3,Toronto Maple Leafs,49,26
4,Florida Panthers,44,30


In [31]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NHL
0,New York City,20153634,RangersIslandersDevils
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


# Asigning the Metropolitan Area to each team

In [40]:
nhl_df['team_ville'] = nhl_df['team']
nhl_df['team_ville'] = nhl_df['team_ville'].map({'Tampa Bay Lightning':'Tampa Bay Area',
 'Boston Bruins':'Boston',
 'Toronto Maple Leafs':'Toronto',
 'Florida Panthers':'Miami–Fort Lauderdale',
 'Detroit Red Wings':'Detroit',
 'Montreal Canadiens':'Montreal',
 'Ottawa Senators':'Ottawa',
 'Buffalo Sabres':'Buffalo',
 'Washington Capitals':'Washington, D.C.',
 'Pittsburgh Penguins':'Pittsburgh',
 'Philadelphia Flyers':'Philadelphia',
 'Columbus Blue Jackets':'Columbus',
 'New Jersey Devils':'New York City',
 'Carolina Hurricanes':'Raleigh',
 'New York Islanders':'New York City',
 'New York Rangers':'New York City',
 'Nashville Predators':'Nashville',
 'Winnipeg Jets':'Winnipeg',
 'Minnesota Wild':'Minneapolis–Saint Paul',
 'Colorado Avalanche':'Denver',
 'St. Louis Blues':'St. Louis',
 'Dallas Stars':'Dallas–Fort Worth',
 'Chicago Blackhawks':'Chicago',
 'Vegas Golden Knights':'Las Vegas',
 'Anaheim Ducks':'Los Angeles',
 'San Jose Sharks':'San Francisco Bay Area',
 'Los Angeles Kings':'Los Angeles',
 'Calgary Flames':'Calgary',
 'Edmonton Oilers':'Edmonton',
 'Vancouver Canucks':'Vancouver',
 'Arizona Coyotes':'Phoenix'})
nhl_df.head()

,team,W,L,team_ville
0,Atlantic Division,Atlantic Division,Atlantic Division,NaN
1,Tampa Bay Lightning,54,23,Tampa Bay Area
2,Boston Bruins,50,20,Boston
3,Toronto Maple Leafs,49,26,Toronto
4,Florida Panthers,44,30,Miami–Fort Lauderdale


In [45]:
nhl_df = nhl_df.dropna(subset=['team_ville'])
nhl_df.head()

,team,W,L,team_ville
1,Tampa Bay Lightning,54,23,Tampa Bay Area
2,Boston Bruins,50,20,Boston
3,Toronto Maple Leafs,49,26,Toronto
4,Florida Panthers,44,30,Miami–Fort Lauderdale
5,Detroit Red Wings,30,39,Detroit


In [50]:
IntTeamsCities = pd.merge(nhl_df,cities,how='inner',left_on='team_ville',right_on='Metropolitan area') # Intersection
IntTeamsCities.head(20)

,team,W,L,team_ville,Metropolitan area,Population (2016 est.)[8],NHL
0,Tampa Bay Lightning,54,23,Tampa Bay Area,Tampa Bay Area,3032171,Lightning
1,Boston Bruins,50,20,Boston,Boston,4794447,Bruins
2,Toronto Maple Leafs,49,26,Toronto,Toronto,5928040,Maple Leafs
3,Florida Panthers,44,30,Miami–Fort Lauderdale,Miami–Fort Lauderdale,6066387,Panthers
4,Detroit Red Wings,30,39,Detroit,Detroit,4297617,Red Wings
5,Montreal Canadiens,29,40,Montreal,Montreal,4098927,Canadiens
6,Ottawa Senators,28,43,Ottawa,Ottawa,1323783,Senators
7,Buffalo Sabres,25,45,Buffalo,Buffalo,1132804,Sabres
8,Washington Capitals,49,26,"Washington, D.C.","Washington, D.C.",6131977,Capitals
9,Pittsburgh Penguins,47,29,Pittsburgh,Pittsburgh,2342299,Penguins


In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values